#### Load packages

In [2]:
import os

from fastai.vision.all import *
from fastai.vision import *

import pandas as pd

#### Create CSV file

Since the [ImageDataLoader](https://docs.fast.ai/vision.data.html#imagedataloaders.from_folder) requires dataset in specific format.
* it needs one class images in to the folder named of that class
* also a `labels.csv` file with two columns `name` and `label` where `name` is the path of the image and `label` is the label.

So, our dataset is in the following format... we need a create a `labels.csv` file in the folder

    dataset
        train 
            class_one
                images
                ...
            class_two
                images
                ...
            labels.csv
        valid
            class_one
                images
                ...
            class_two
                images
                ...
            labels.csv

In [ ]:
def create_labels_csv_file(dataset_path: str) -> None:
    class_names = []
    for folder_name in os.listdir(dataset_path):
        class_names.append(folder_name)

    image_paths = []
    labels = []

    for i in range(len(class_names)):
        class_name = class_names[i]
        if '.csv' in os.path.basename(class_name):
            continue
        image_names = os.listdir(
            os.path.join(
                dataset_path,
                class_name
            )
        )
        for image_name in image_names:
            image_path = os.path.join(
                dataset_path,
                class_name,
                image_name
            )
            image_paths.append(image_path)
            labels.append(i)

    df = pd.DataFrame(
        {
            'name': image_paths,
            'label': labels
        }
    )

    df.to_csv(
        path_or_buf=os.path.join(
            dataset_path,
            'labels.csv'
        ),
        index=False
    )

In [ ]:
create_labels_csv_file(
    dataset_path=os.path.join(
        'dataset',
        'train'
    )
)

create_labels_csv_file(
    dataset_path=os.path.join(
        'dataset',
        'valid'
    )
)

#### Load the data

In [ ]:
dls = ImageDataLoaders.from_folder(
    path='dataset',
    train='train',
    valid='valid',
    item_tfms=Resize(224),
    bs=16,
    batch_tfms=[Normalize.from_stats(*imagenet_stats), RandTransform()]
)

#### Let's see the images

In [ ]:
dls.valid.show_batch()

In [ ]:
dls.train.show_batch()

#### Build a model

In [ ]:
model = vision_learner(
    dls=dls,
    arch=models.resnet50,
    metrics=[accuracy, error_rate]
)

#### Train the model

In [ ]:
model.fine_tune(
    epochs=1
)

#### Let's see the results

In [ ]:
model.show_results()

In [ ]:
model.validate()

#### Save the model

In [ ]:
model.export('intel_scene_classifier.pkl')

#### Load the model

In [ ]:
new_model = load_learner('dataset\intel_scene_classifier.pkl')

#### Predict on a single image

In [ ]:
new_model.predict(
    item='dataset/test/test/24298.jpg'
)